# Data Cleanup of Heart Disease Dataset

---

This Jupyter Notebook was created for the **CardioVision** project by *ClinData Solutions*.

### Prerequisite
The original raw data needs to be combined into a single CSV file, including the fixed Cleveland data.

### Objectives
This Jupyter Notebook focuses on:
- Preparing raw data for the **Exploratory Data Analysis (EDA)**
- Preparing raw data for **predictive machine learning**

---

### Acknowledgements
The authors of the databases have requested that any publications resulting from the use of the data include the names of the principal investigators responsible for the data collection at each institution:

1. **Hungarian Institute of Cardiology, Budapest:** *Andras Janosi, M.D.*
2. **University Hospital, Zurich, Switzerland:** *William Steinbrunn, M.D.*
3. **University Hospital, Basel, Switzerland:** *Matthias Pfisterer, M.D.*
4. **V.A. Medical Center, Long Beach and Cleveland Clinic Foundation:** *Robert Detrano, M.D., Ph.D.*

### Original Publication
The first publication that used this dataset is:

*Detrano R, Janosi A, Steinbrunn W, Pfisterer M, Schmid JJ, Sandhu S, et al. International application of a new probability algorithm for the diagnosis of coronary artery disease. The American Journal of Cardiology. 1989; 64(5):304–10.*

---

# Table of contents

Following parts are covered in this Jupyter Notebook:
1. Importing the raw data csv
2. Selecting relevant features based on Expert opinion
3. Handling Feature Categories
4. Handling Null-Values and Duplicates

## 0. Preparations

Load all needed libraries used in this Jupyter Notebook.

In [1]:
import pandas as pd
import numpy as np

## 1. Importing the raw data csv

In [2]:
# Load the "combined_data_all.csv" and save it into a dataframe
combined_data_all_df = pd.read_csv("combined_data_all.csv")

# Show the dataframe
combined_data_all_df

,Dataset,id,ccf,age,sex,painloc,painexer,relrest,pncaden,cp,...,rcaprox,rcadist,lvx1,lvx2,lvx3,lvx4,lvf,cathef,junk,name
0,Cleveland,1,0,63,1,-9,-9,-9,-9,1,...,1,1,1,1,1,1,1,-9.00,-9.0,name
1,Cleveland,2,0,67,1,-9,-9,-9,-9,4,...,1,1,1,1,1,1,1,-9.00,-9.0,name
2,Cleveland,3,0,67,1,-9,-9,-9,-9,4,...,2,2,1,1,1,7,3,-9.00,-9.0,name
3,Cleveland,4,0,37,1,-9,-9,-9,-9,3,...,1,1,1,1,1,1,1,-9.00,-9.0,name
4,Cleveland,6,0,41,0,-9,-9,-9,-9,2,...,1,1,1,1,1,1,1,-9.00,-9.0,name
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,Long Beach VA,200,0,54,0,1,1,1,-9,4,...,1,1,1,1,1,1,1,0.76,5.6,name
895,Long Beach VA,201,0,62,1,0,0,0,-9,1,...,1,1,1,1,1,1,2,0.62,3.5,name
896,Long Beach VA,202,0,55,1,1,1,1,-9,4,...,2,1,1,1,1,1,1,0.69,5.6,name
897,Long Beach VA,116,0,58,1,1,1,1,-9,4,...,1,1,1,1,1,1,1,0.81,6.0,name


## 2. Selection of Chosen Variables

The original dataset contains 76 features, but only 14 have been widely used in publications so far. Some features describe others; for instance, the boolean "hypertension" feature indicates whether the resting blood pressure value is above or below a certain threshold.

All 14 commonly used features were also identified by our domain expert as they are known risk factors for heart disease. In addition, 4 more features were selected for inclusion in the initial Exploratory Data Analysis (EDA).

### Selected Features Table

| **No.** | **Feature Name** | **Description**                                            | **Feature Type**         |
|---------|------------------|----------------------------------------------------------|--------------------------|
| 1       | age              | Age of the patient                                        | Numerical                |
| 2       | sex              | Sex of the patient (1 = male; 0 = female)                 | Categorical (Binary)     |
| 3       | cp               | Chest pain type (1 = typical angina; 2 = atypical angina; 3 = non-anginal pain; 4 = asymptomatic) | Categorical              |
| 4       | chol             | Serum cholesterol in mg/dl                                | Numerical                |
| 5       | fbs              | Fasting blood sugar > 120 mg/dl (1 = true; 0 = false)     | Categorical (Binary)     |
| 6       | restecg          | Resting electrocardiographic results                      | Categorical              |
| 7       | thalach          | Maximum heart rate achieved                               | Numerical                |
| 8       | exang            | Exercise-induced angina (1 = yes; 0 = no)                 | Categorical (Binary)     |
| 9       | oldpeak          | ST depression induced by exercise relative to rest       | Numerical                |
| 10      | slope            | The slope of the peak exercise ST segment (1 = upsloping; 2 = flat; 3 = downsloping) | Categorical              |
| 11      | ca               | Number of major vessels (0-3) colored by fluoroscopy     | Numerical (Discrete)     |
| 12      | painloc          | Chest pain location (1 = substernal; 0 = otherwise)       | Categorical (Binary)     |
| 13      | htn              | History of hypertension (1 = yes; 0 = no)                 | Categorical (Binary)     |
| 14      | cigs             | Number of cigarettes smoked per day                      | Numerical                |
| 15      | years            | Number of years as a smoker                              | Numerical                |
| 16      | famhist          | Family history of coronary artery disease (1 = yes; 0 = no) | Categorical (Binary)  |
| 17      | thalrest         | Resting heart rate                                       | Numerical                |
| 18      | num              | Diagnosis of heart disease (0-4, representing increasing severity) | Categorical (Ordinal)   |

---





In [3]:
# Create a new df, that keeps only the selected columns.
selected_df = combined_data_all_df[["age", "sex", "cp", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "painloc", "htn", 
 "cigs", "years", "famhist", "thalrest", "num"]]

# Display the new df.
selected_df

,age,sex,cp,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,painloc,htn,cigs,years,famhist,thalrest,num
0,63,1,1,233,1,2,150,0,2.3,3,0,-9,1,50,20,1,60,0
1,67,1,4,286,0,2,108,1,1.5,2,3,-9,1,40,40,1,64,2
2,67,1,4,229,0,2,129,1,2.6,2,2,-9,1,20,35,1,78,1
3,37,1,3,250,0,0,187,0,3.5,3,0,-9,0,0,0,1,84,0
4,41,0,2,204,0,2,172,0,1.4,1,0,-9,1,0,0,1,71,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,54,0,4,333,1,1,154,0,0.0,-9,-9,1,0,0,0,1,83,1
895,62,1,1,139,0,1,-9,-9,-9.0,-9,-9,0,0,15,30,0,-9,0
896,55,1,4,223,1,1,100,0,0.0,-9,-9,1,1,20,40,0,74,2
897,58,1,4,385,1,2,-9,-9,-9.0,-9,-9,1,0,10,20,1,-9,0


## 3. Handling Feature Categories

Check all datatypes of the features and change them accordingly to the table above.

In [4]:
# Check the datatypes of the selected data
selected_df.dtypes

age           int64
sex           int64
cp            int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
painloc       int64
htn           int64
cigs          int64
years         int64
famhist       int64
thalrest      int64
num           int64
dtype: object

The datatype of some features is int64 instead of categorical, as mentioned in the table above. 
They should be changed to optimize memory usage and for the machine learning algorithms.

!!! Attention !!!
The dataset uses -9 for NaN values. These should be replaced by NaN again, as numpy and pandas are working with NaN natively.
This needs to be done before changing the datatype to categorical.

In [5]:
# Replace -9 with NaN
selected_df = selected_df.replace(-9, np.nan)

In [6]:
selected_df

,age,sex,cp,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,painloc,htn,cigs,years,famhist,thalrest,num
0,63,1,1,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,NaN,1.0,50.0,20.0,1.0,60.0,0
1,67,1,4,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,NaN,1.0,40.0,40.0,1.0,64.0,2
2,67,1,4,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,NaN,1.0,20.0,35.0,1.0,78.0,1
3,37,1,3,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,NaN,0.0,0.0,0.0,1.0,84.0,0
4,41,0,2,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,NaN,1.0,0.0,0.0,1.0,71.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,54,0,4,333.0,1.0,1.0,154.0,0.0,0.0,NaN,NaN,1.0,0.0,0.0,0.0,1.0,83.0,1
895,62,1,1,139.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,15.0,30.0,0.0,NaN,0
896,55,1,4,223.0,1.0,1.0,100.0,0.0,0.0,NaN,NaN,1.0,1.0,20.0,40.0,0.0,74.0,2
897,58,1,4,385.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,10.0,20.0,1.0,NaN,0


In [7]:
# Convert columns to categorical data types, handling NaN values properly
columns_to_convert = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'painloc', 'htn', 'famhist']

for col in columns_to_convert:
    selected_df[col] = selected_df[col].astype("category")

# Display the data types to confirm the changes
print(selected_df.dtypes)

age            int64
sex         category
cp          category
chol         float64
fbs         category
restecg     category
thalach      float64
exang       category
oldpeak      float64
slope       category
ca          category
painloc     category
htn         category
cigs         float64
years        float64
famhist     category
thalrest     float64
num            int64
dtype: object


## 4. Handle missing and duplicated values

The Null/ NaN values were originally replaced by -9. Before converting the to categorical features, the -9 was again replaced by NaN so that the built-in functions are working again.

In [8]:
# Calculate the total number of missing values per feature
missing_values_total = selected_df.isnull().sum()

# Calculate the percentage of missing values per feature
missing_values_percentage = (missing_values_total / len(selected_df)) * 100

# Create a DataFrame to display both total and percentage of missing values per feature
missing_values_table = pd.DataFrame({
    'Total Missing': missing_values_total,
    'Percentage Missing (%)': missing_values_percentage
})

# Display the table
print(missing_values_table)

          Total Missing  Percentage Missing (%)
age                   0                0.000000
sex                   0                0.000000
cp                    0                0.000000
chol                 30                3.337041
fbs                  90               10.011123
restecg               2                0.222469
thalach              55                6.117909
exang                55                6.117909
oldpeak              62                6.896552
slope               308               34.260289
ca                  608               67.630701
painloc             282               31.368187
htn                  34                3.781980
cigs                420               46.718576
years               432               48.053393
famhist             422               46.941046
thalrest             56                6.229143
num                   0                0.000000


### Handle Null Values

We need to find a way to handle the features with high amounts of null values such as ca, cigs and so so on...

In [9]:
# Check the dataset for duplicates, if false continue
selected_df.duplicated().any()

np.True_

In [10]:
# Identify the duplicate row
duplicate_row = selected_df[selected_df.duplicated(keep=False)]

# Display the rows that are identical
print(duplicate_row)

     age sex cp   chol  fbs restecg  thalach exang  oldpeak slope  ca painloc  \
838   58   1  3  219.0  0.0     1.0    118.0   1.0      0.0   NaN NaN     1.0   
886   58   1  3  219.0  0.0     1.0    118.0   1.0      0.0   NaN NaN     1.0   

     htn  cigs  years famhist  thalrest  num  
838  1.0  20.0   35.0     1.0      70.0    2  
886  1.0  20.0   35.0     1.0      70.0    2  


Is it likely that these values might be the same on two patients? Otherwise describe here that it was dropped or kept.

In [11]:
# Perform full descriptive analysis of the dataset to check if all works
selected_df.describe(include='all')

,age,sex,cp,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,painloc,htn,cigs,years,famhist,thalrest,num
count,899.000000,899.0,899.0,869.000000,809.0,897.0,844.000000,844.0,837.000000,591.0,291.0,617.0,865.0,479.000000,467.000000,477.0,843.000000,899.000000
unique,NaN,2.0,4.0,NaN,2.0,3.0,NaN,2.0,NaN,4.0,5.0,2.0,2.0,NaN,NaN,2.0,NaN,NaN
top,NaN,1.0,4.0,NaN,0.0,0.0,NaN,0.0,NaN,2.0,0.0,1.0,0.0,NaN,NaN,1.0,NaN,NaN
freq,NaN,711.0,485.0,NaN,674.0,538.0,NaN,514.0,NaN,334.0,171.0,568.0,453.0,NaN,NaN,269.0,NaN,NaN
mean,53.480534,NaN,NaN,198.759494,NaN,NaN,137.298578,NaN,0.870490,NaN,NaN,NaN,NaN,19.118998,18.796574,NaN,75.487544,1.129032
std,9.435894,NaN,NaN,111.834415,NaN,NaN,25.965959,NaN,1.080548,NaN,NaN,NaN,NaN,18.296273,16.359145,NaN,14.727961,1.259720
min,28.000000,NaN,NaN,0.000000,NaN,NaN,60.000000,NaN,-2.600000,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,37.000000,0.000000
25%,47.000000,NaN,NaN,175.000000,NaN,NaN,120.000000,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,65.000000,0.000000
50%,54.000000,NaN,NaN,224.000000,NaN,NaN,140.000000,NaN,0.500000,NaN,NaN,NaN,NaN,20.000000,20.000000,NaN,74.000000,1.000000
75%,60.000000,NaN,NaN,269.000000,NaN,NaN,157.000000,NaN,1.500000,NaN,NaN,NaN,NaN,30.000000,30.000000,NaN,84.000000,2.000000


## 5. Save the final df to a csv file that is ready to be used for the EDA and machine learning

In [12]:
selected_df.to_csv("dataset_with_selected_features.csv", index=False)